Paillier's encryption Scheme
==========================

<img width=60% src="https://vitalik.eth.limo/images/stealth/stealth_workflow.png"/>

In [34]:
from zkp_playground.algebra.concrete import EllipticCurveGroupSecp256k1 as Curve
from zkp_playground.algebra.concrete import FiniteFieldCyclicSecp256k1 as F
from zkp_playground.algebra.utils import randfield
from zkp_playground.blockchain.ethereum.address import gen_address as addr
G = Curve.G


#### <i>1. Bob generates his root spending key (m) and stealth meta-address (M).</i>

In [19]:
m = randfield(F)
M = G @ m

In [20]:
m, M

(FiniteFieldCyclicSecp256k1::0x6725835acb15b840222bf4fa44ce6d99a214aec4d2ee2efe64ce04bd89b8d55c,
 EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::0xde7b5f6b84fd039a79d4e70df49b174f9aaa89189524fda45f37662542e099c7, FiniteFieldSecp256k1::0x4a0ddcc995127c89ad532c29228a3c16f75afc497d94db38c20c315e20961192))

#### 2. Bob adds an ENS record to register M as the stealth meta-address for bob.eth.
#### 3. We assume Alice knows that Bob is bob.eth. Alice looks up his stealth meta-address M on ENS.
#### 4. <i>Alice generates an ephemeral key that only she knows, and that she only uses once (to generate this specific stealth address).</i>

In [21]:
r = randfield(F)

#### <i>5. Alice uses an algorithm that combines her ephemeral key and Bob's meta-address to generate a stealth address. She can now send assets to this address.</i>

##### Stealth addresses generation.

* Bob generates a key m, and computes M = G * m, where G is a commonly-agreed generator point for the elliptic curve. The stealth meta-address is an encoding of M.

#### 6. Alice also generates her ephemeral public key, and publishes it to the ephemeral public key registry (this can be done in the same transaction as the first transaction sending assets to this stealth address).


* Alice generates an ephemeral key r, and publishes the ephemeral public key R = G * r.



In [22]:
R = G @ r

* Alice can compute a shared secret S = M * r, and Bob can compute the same shared secret S = m * R.



In [24]:
S = M @ r

In [25]:
S == M @ r == R @ m

True

* In general, in both Bitcoin and Ethereum (including correctly-designed ERC-4337 accounts), an address is a hash containing the public key used to verify transactions from that address. So you can compute the address if you compute the public key. To compute the public key, Alice or Bob can compute P = M + G * hash(S)



In [44]:
hash_s = F(int(addr(S), 16))

In [45]:
P = M + G @ hash_s

* To compute the private key for that address, Bob (and Bob alone) can compute p = m + hash(S)

#### 7. For Bob to discover stealth addresses belonging to him, Bob needs to scan the ephemeral public key registry for the entire list of ephemeral public keys published by anyone for any reason since the last time he did the scan.

In [46]:
p = m + hash_s

In [48]:
G @ p == P

True

#### 8.     For each ephemeral public key, Bob attempts to combine it with his root spending key to generate a stealth address, and checks if there are any assets in that address. If there are, Bob computes the spending key for that address and remembers it.



### Together

In [1]:
from zkp_playground.algebra.concrete import EllipticCurveGroupSecp256k1 as Curve
from zkp_playground.algebra.concrete import FiniteFieldCyclicSecp256k1 as F
from zkp_playground.algebra.utils import randfield
from zkp_playground.blockchain.ethereum.address import gen_address as addr
G = Curve.G
m = randfield(F)
M = G @ m
r = randfield(F)
R = G @ r
S = M @ r
assert S == M @ r == R @ m
hash_s = F(int(addr(S), 16))
P = M + G @ hash_s
p = m + hash_s
assert G @ p == P